In [1]:
import io

import folium
import imageio
import numpy as np
import pandas as pd
from folium.features import DivIcon
from folium.plugins import HeatMap
from PIL import Image
from selenium import webdriver

In [2]:
df = pd.read_csv("date-hour-soo-dest-2019.csv")

In [3]:
df.columns = [x.lower() for x in df.columns]
df = df.rename(
    columns={
        "origin station": "origin",
        "destination station": "destination",
        "trip count": "count",
    }
)

In [4]:
df_origin = df.groupby(by=["hour", "origin"]).agg("sum").reset_index()
df_destination = df.groupby(by=["hour", "destination"]).agg("sum").reset_index()

/var/folders/59/p50xxbtd4y5f88p88xxyc8bw0000gn/T/ipykernel_5098/3417220881.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_origin = df.groupby(by=['hour', 'origin']).agg('sum').reset_index()
/var/folders/59/p50xxbtd4y5f88p88xxyc8bw0000gn/T/ipykernel_5098/3417220881.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_destination = df.groupby(by=['hour', 'destination']).agg('sum').reset_index()


In [5]:
print(df_origin.origin.nunique())
print(df_destination.destination.nunique())

50
50


In [6]:
df_origin.head()

,hour,origin,count
0,0,12TH,15073
1,0,16TH,49144
2,0,19TH,28573
3,0,24TH,22558
4,0,ANTC,1709


In [7]:
df_origin_destination = pd.merge(
    df_origin,
    df_destination,
    left_on=["hour", "origin"],
    right_on=["hour", "destination"],
)

In [8]:
df_origin_destination = df_origin_destination.rename(
    columns={"count_x": "count_in", "count_y": "count_out"}
)
df_origin_destination

,hour,origin,count_in,destination,count_out
0,0,12TH,15073,12TH,23018
1,0,16TH,49144,16TH,19907
2,0,19TH,28573,19TH,21108
3,0,24TH,22558,24TH,27375
4,0,ANTC,1709,ANTC,14458
...,...,...,...,...,...
1169,23,UCTY,6188,UCTY,27462
1170,23,WARM,10833,WARM,27241
1171,23,WCRK,13784,WCRK,33237
1172,23,WDUB,4870,WDUB,25029


In [9]:
df_station = pd.read_csv("BART_Station.csv")

In [10]:
df_station = df_station.rename(
    columns={"X": "long", "Y": "lat", "Name": "station", "Abbr": "abbr"}
)
df_station.head()

,long,lat,station,abbr
0,-121.891094,37.410282,Milpitas,MLPT
1,-121.874694,37.368478,Berryessa/North San Jose,BERY
2,-122.413936,37.779394,Civic Center/UN Plaza,CIVC
3,-122.029062,37.973825,Concord,CONC
4,-122.466223,37.684622,Colma,COLM


In [11]:
df_merge = pd.merge(
    df_origin_destination, df_station, left_on="origin", right_on="abbr", how="left"
)
df_merge.head()

,hour,origin,count_in,destination,count_out,long,lat,station,abbr
0,0,12TH,15073,12TH,23018,-122.271676,37.803590,12th St/Oakland City Center,12TH
1,0,16TH,49144,16TH,19907,-122.419708,37.765054,16th St/Mission,16TH
2,0,19TH,28573,19TH,21108,-122.268654,37.808493,19th St/Oakland,19TH
3,0,24TH,22558,24TH,27375,-122.418468,37.752231,24th St/Mission,24TH
4,0,ANTC,1709,ANTC,14458,-121.780433,37.995393,Antioch,ANTC


In [12]:
df_merge["net"] = df_merge["count_out"] - df_merge["count_in"]
df_merge["color"] = ["red" if x >= 0 else "green" for x in df_merge["net"]]

In [13]:
df_merge

,hour,origin,count_in,destination,count_out,long,lat,station,abbr,net,color
0,0,12TH,15073,12TH,23018,-122.271676,37.803590,12th St/Oakland City Center,12TH,7945,red
1,0,16TH,49144,16TH,19907,-122.419708,37.765054,16th St/Mission,16TH,-29237,green
2,0,19TH,28573,19TH,21108,-122.268654,37.808493,19th St/Oakland,19TH,-7465,green
3,0,24TH,22558,24TH,27375,-122.418468,37.752231,24th St/Mission,24TH,4817,red
4,0,ANTC,1709,ANTC,14458,-121.780433,37.995393,Antioch,ANTC,12749,red
...,...,...,...,...,...,...,...,...,...,...,...
1169,23,UCTY,6188,UCTY,27462,-122.017256,37.590734,Union City,UCTY,21274,red
1170,23,WARM,10833,WARM,27241,-121.939433,37.502335,Warm Springs/South Fremont,WARM,16408,red
1171,23,WCRK,13784,WCRK,33237,-122.067329,37.905814,Walnut Creek,WCRK,19453,red
1172,23,WDUB,4870,WDUB,25029,-121.928077,37.699751,West Dublin/Pleasanton,WDUB,20159,red


# Plotting - Colored circle Map Animation

In [23]:
def make_ccmap(df, time):
    df_zoom = df[df.hour == time]

    latitude = 37.759894
    longitude = -122.285457

    # Create map and display it
    fig = folium.Figure(width=700, height=500)

    sf_map = folium.Map(
        width="%100",
        height="%100",
        location=[latitude, longitude],
        zoom_start=10.2,
        tiles="cartodbpositron",
    ).add_to(fig)

    for name, row in df_zoom.iterrows():
        radius = row["net"] / 250
        folium.Circle(
            location=(row["lat"], row["long"]),
            radius=radius,
            color=row["color"],
            fill=True,
            fill_opacity=0.1,
        ).add_to(sf_map)

    folium.map.Marker(
        [37.936835, -122.549275],
        icon=DivIcon(
            icon_size=(250, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 25pt">{time}:00</div>',
        ),
    ).add_to(sf_map)
    folium.map.Marker(
        [38.025940, -122.549275],
        icon=DivIcon(
            icon_size=(250, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 30pt">BART</div>',
        ),
    ).add_to(sf_map)
    folium.map.Marker(
        [37.524138, -122.549275],
        icon=DivIcon(
            icon_size=(250, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12pt; color:red">Red: Net Inflow</div>',
        ),
    ).add_to(sf_map)
    folium.map.Marker(
        [37.464337, -122.549275],
        icon=DivIcon(
            icon_size=(250, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12pt; color:green">Green: Net Outflow</div>',
        ),
    ).add_to(sf_map)
    folium.map.Marker(
        [37.464337, -121.805980],
        icon=DivIcon(
            icon_size=(250, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12pt">Credit: Chung-Hao Lee</div>',
        ),
    ).add_to(sf_map)

    sf_map

    img_data = sf_map._to_png(3)
    img = Image.open(io.BytesIO(img_data))
    img.save(f"{time}.png")

In [24]:
for i in df_merge.hour.unique():
    make_ccmap(df_merge, i)

In [25]:
with imageio.get_writer(uri="BART_animation.gif", mode="I", fps=2) as writer:
    for i in df_merge.hour.unique():
        writer.append_data(imageio.imread(f"{i}.png"))

/var/folders/59/p50xxbtd4y5f88p88xxyc8bw0000gn/T/ipykernel_5098/776227436.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  writer.append_data(imageio.imread(f"{i}.png"))
